In [1]:
import os
import copy
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from scipy.stats import multivariate_normal

from microsim.population import (NHANESDirectSamplePopulation, 
                                 build_people_using_nhanes_for_sampling,
                                 microsimToNhanes, Population, get_nhanes_population)
from microsim.risk_model_repository import RiskModelRepository
from microsim.gender import NHANESGender
from microsim.smoking_status import SmokingStatus
from microsim.race_ethnicity import NHANESRaceEthnicity
from microsim.education import Education
from microsim.treatment import DefaultTreatmentsType,  TreatmentStrategiesType
from microsim.risk_factor import StaticRiskFactorsType, DynamicRiskFactorsType
from microsim.afib_model import AFibPrevalenceModel
from microsim.pvd_model import PVDPrevalenceModel
from microsim.outcome import Outcome, OutcomeType
from microsim.person import Person
from microsim.population_model_repository import PopulationModelRepository
from microsim.cohort_risk_model_repository import (CohortStaticRiskFactorModelRepository,
                                                   CohortDynamicRiskFactorModelRepository,
                                                   CohortDefaultTreatmentModelRepository)
from microsim.outcome_model_repository import OutcomeModelRepository

microsimDir = "/Users/deligkaris.1/OneDrive - The Ohio State University Wexner Medical Center/MICROSIM"
os.chdir(microsimDir+"/CODE/microsim")

dataDir = microsimDir+"/NOTEBOOKS/DATA"

popSize = 300000

In [2]:
categoricalVars = ["gender", "smokingStatus", "raceEthnicity", "statin",'education',
                  'alcoholPerWeek','anyPhysicalActivity','antiHypertensiveCount']
continuousVars = ['age', 'hdl', 'bmi', 'totChol', 'trig', 'a1c', 'ldl', 'waist', 'creatinine', 'sbp', 'dbp']

In [3]:
nhanesDf = pd.read_stata(microsimDir + "/CODE/microsim/microsim/data/fullyImputedDataset.dta")
nhanesDf = nhanesDf.rename(columns={"level_0":"name"})

In [4]:
df = pd.read_csv(dataDir+"/nhanes-persons-from-Gaussians.csv")
df.head()

,name,gender,smokingStatus,raceEthnicity,statin,education,alcoholPerWeek,anyPhysicalActivity,antiHypertensiveCount,age,hdl,bmi,totChol,trig,a1c,ldl,waist,creatinine,sbp,dbp
0,41909,1,0,1,False,1,0,0,0.0,40,59.860116,30.433631,243.771416,60.782933,5.915046,154.068838,98.907992,0.859896,117.463015,80.630951
1,42688,1,0,1,False,1,0,0,0.0,66,49.866495,26.274580,252.707847,93.523817,6.556322,170.343466,94.925142,0.906850,144.930624,78.151846
2,43025,1,0,1,False,1,0,0,0.0,58,51.263620,28.790801,207.808584,168.141061,6.219358,122.685090,100.727180,0.954121,126.312875,68.255347
3,43390,1,0,1,False,1,0,0,0.0,41,53.007622,29.412992,228.923683,65.223746,5.654073,144.108985,100.058501,0.775893,127.072760,76.350982
4,44501,1,0,1,False,1,0,0,0.0,51,53.265911,21.477110,179.991013,119.950624,5.342257,110.602062,80.954864,0.631021,129.142989,69.216155


In [5]:
df["age"].describe()

count    5448.000000
mean       43.831131
std        17.465267
min        13.000000
25%        30.000000
50%        43.000000
75%        56.000000
max       110.000000
Name: age, dtype: float64

In [6]:
df.shape

(5448, 20)

In [7]:
df = df.merge(nhanesDf[["name","WTINT2YR"]], on="name", how="inner").copy()
df.shape

(5448, 21)

In [8]:
def build_person(x, initializationModelRepository):
    """Takes all Person-instance-related data via x and initializationModelRepository and organizes it,
       passes the organized data to the Person class and returns a Person instance."""
    
    rng = np.random.default_rng()
        
    name = x.name
    
    personStaticRiskFactors = {
                        StaticRiskFactorsType.RACE_ETHNICITY.value: NHANESRaceEthnicity(int(x.raceEthnicity)),
                        StaticRiskFactorsType.EDUCATION.value: Education(int(x.education)),
                        StaticRiskFactorsType.GENDER.value: NHANESGender(int(x.gender)),
                        StaticRiskFactorsType.SMOKING_STATUS.value: SmokingStatus(int(x.smokingStatus))}
        
    #use this to get the bounds imposed on the risk factors in a bit
    rfRepository = RiskModelRepository()
        
    #TO DO: find a way to include everything here, including the rfs that need initialization
    #the PVD model would be easy to implement, eg with an estimate_next_risk_for_patient_characteristics function
    #but the AFIB model would be more difficult because it relies on statsmodel_logistic_risk file
    #for now include None, in order to create the risk factor lists correctly at the Person instance
    personDynamicRiskFactors = dict()
    for rfd in DynamicRiskFactorsType:
        #if rfd==DynamicRiskFactorsType.ALCOHOL_PER_WEEK:
        #    personDynamicRiskFactors[rfd.value] = AlcoholCategory.get_category_for_consumption(rfRepository.apply_bounds(rfd.value, x[microsimToNhanes[rfd.value]]))
        #else:
        if (rfd!=DynamicRiskFactorsType.PVD) & (rfd!=DynamicRiskFactorsType.AFIB):
            personDynamicRiskFactors[rfd.value] = rfRepository.apply_bounds(rfd.value, x[rfd.value])
    personDynamicRiskFactors[DynamicRiskFactorsType.AFIB.value] = None
    personDynamicRiskFactors[DynamicRiskFactorsType.PVD.value] = None

    #Q: do we need otherLipid treatment? I am not bringing it to the Person objects for now.
    #A: it is ok to leave it out as we do not have a model to update this. It is also very rarely taking place in the population anyway.
    #also: used to have round(x.statin) but NHANES includes statin=2...
    personDefaultTreatments = {
                        DefaultTreatmentsType.STATIN.value: bool(x.statin),
                        #DefaultTreatmentsType.OTHER_LIPID_LOWERING_MEDICATION_COUNT.value: x.otherLipidLowering,
                        DefaultTreatmentsType.ANTI_HYPERTENSIVE_COUNT.value: x.antiHypertensiveCount}

    personTreatmentStrategies = dict(zip([strategy.value for strategy in TreatmentStrategiesType],
                                         #[None for strategy in range(len(TreatmentStrategiesType))]))
                                         [{"status": None} for strategy in range(len(TreatmentStrategiesType))]))

    personOutcomes = dict(zip([outcome for outcome in OutcomeType],
                                  [list() for outcome in range(len(OutcomeType))]))
    #add pre-simulation stroke outcomes
    #selfReportStrokeAge=x.selfReportStrokeAge
    #Q: we should not add the stroke outcome in case of "else"?
    #if selfReportStrokeAge is not None and selfReportStrokeAge > 1:
    #        selfReportStrokeAge = selfReportStrokeAge if selfReportStrokeAge <= x.age else x.age
    #        personOutcomes[OutcomeType.STROKE].append((selfReportStrokeAge, StrokeOutcome(False, None, None, None, priorToSim=True)))
    #add pre-simulation mi outcomes
    #selfReportMIAge=rng.integers(18, x.age) if x.selfReportMIAge == 99999 else x.selfReportMIAge
    #if selfReportMIAge is not None and selfReportMIAge > 1:
    #        selfReportMIAge = selfReportMIAge if selfReportMIAge <= x.age else x.age
    #        personOutcomes[OutcomeType.MI].append((selfReportMIAge, Outcome(OutcomeType.MI, False, priorToSim=True)))

    person = Person(name,
                   personStaticRiskFactors,
                   personDynamicRiskFactors,
                   personDefaultTreatments,
                   personTreatmentStrategies,
                   personOutcomes)

    #TO DO: find a way to initialize these rfs above with everything else
    person._pvd = [initializationModelRepository[DynamicRiskFactorsType.PVD].estimate_next_risk(person)]
    person._afib = [initializationModelRepository[DynamicRiskFactorsType.AFIB].estimate_next_risk(person)]
    return person


In [9]:
initializationModelRepository = {DynamicRiskFactorsType.AFIB: AFibPrevalenceModel(), 
                                     DynamicRiskFactorsType.PVD: PVDPrevalenceModel()}

In [10]:
dfSample = df.sample(popSize, weights=df.WTINT2YR, replace=True)

In [11]:
people = dfSample.apply(lambda x: build_person(x, initializationModelRepository), axis=1)

In [12]:
#sets the unique identifier for each Person instance
noneList = list(map(lambda person, i: setattr(person, "_index", i), people, range(len(people)))) 

In [13]:
popModelRepository = PopulationModelRepository(CohortDynamicRiskFactorModelRepository(),
                                                           CohortDefaultTreatmentModelRepository(),
                                                           OutcomeModelRepository(),
                                                           CohortStaticRiskFactorModelRepository()) 
pop = Population(people, popModelRepository)

In [14]:
pop._people.shape

(300000,)

In [15]:
#pop2 = get_nhanes_population(1999)
pop2 = NHANESDirectSamplePopulation(popSize, 1999)
pop2.advance(1)
#pop2.print_baseline_summary()

In [16]:
pop.print_baseline_summary_comparison(pop2)

                                                     self                              other
                                                     min    0.25   med    0.75    max   min    0.25   med    0.75    max
                                               age   18.0   31.0   44.2   55.0   85.0   18.0   30.0   43.8   55.0   85.0
                                               sbp   72.0  111.0  122.5  133.0  219.0   72.0  109.0  122.3  131.0  266.0
                                               dbp   40.0   64.0   71.5   79.0  116.0   40.0   65.0   72.1   79.0  132.0
                                               a1c    3.0    4.0    4.9    5.0   11.0    2.0    5.0    5.0    5.0   15.0
                                               hdl   18.0   42.0   51.8   60.0  125.0    8.0   40.0   50.8   59.0  151.0
                                               ldl   29.0  103.0  126.0  147.0  259.0   28.0   98.0  123.0  145.0  354.0
                                              trig   20.0   

In [17]:
personsOut = list(filter( lambda x: ((x._sbp[0]>=72.) & (x._sbp[0]<=81.)) | ((x._sbp[0]>=207.) & (x._sbp[0]<=266.)) , pop2._people))

In [18]:
len(personsOut)

823

In [19]:
personsOutUnique = set(map( lambda x: x._name, personsOut) )

In [20]:
len(personsOutUnique)

21

In [21]:
%%time
pop.advance(18, None, nWorkers=5)

CPU times: user 1min 40s, sys: 2min 53s, total: 4min 33s
Wall time: 31min 37s


In [ ]:
%%time
pop2.advance(18, None, nWorkers=5)

In [ ]:
pop.print_lastyear_summary_comparison(pop2)

In [ ]:
%%time
pop.print_cv_standardized_rates()

In [ ]:
%%time
pop2.print_cv_standardized_rates()

In [ ]:
%%time
pop.print_dementia_incidence()

In [ ]:
nYears=5
nSimulations = 4
for bpMedsAdded in [1]:
    miRRList = list()
    strokeRRList = list()
    print(f"\nbpMedsAdded={bpMedsAdded}")
    for i in range(nSimulations):
        treatmentPop = NHANESDirectSamplePopulation(popSize, 1999)
        controlPop = NHANESDirectSamplePopulation(popSize, 1999)
        treatmentStrategies = TreatmentStrategyRepository()
        #treatmentStrategies._repository[TreatmentStrategiesType.BP.value] = AddASingleBPMedTreatmentStrategy()
        treatmentStrategies._repository[TreatmentStrategiesType.BP.value] = AddNBPMedsTreatmentStrategy(bpMedsAdded)
        treatmentPop.advance_parallel(1, treatmentStrategies = treatmentStrategies, nWorkers=nWorkers)
        treatmentStrategies._repository[TreatmentStrategiesType.BP.value].status = TreatmentStrategyStatus.MAINTAIN
        treatmentPop.advance_parallel(nYears-1, treatmentStrategies = treatmentStrategies, nWorkers=nWorkers)
        controlPop.advance_parallel(nYears, nWorkers=nWorkers)
        controlRisk = sum(list(map(lambda x: x.has_outcome_during_simulation(OutcomeType.STROKE),
                                       controlPop._people)))/controlPop._n
        treatmentRisk = sum(list(map(lambda x: x.has_outcome_during_simulation(OutcomeType.STROKE),
                                         treatmentPop._people)))/treatmentPop._n
        strokeRR = treatmentRisk/controlRisk
        controlRisk = sum(list(map(lambda x: x.has_outcome_during_simulation(OutcomeType.MI),
                                       controlPop._people)))/controlPop._n
        treatmentRisk = sum(list(map(lambda x: x.has_outcome_during_simulation(OutcomeType.MI),
                                         treatmentPop._people)))/treatmentPop._n
        miRR = treatmentRisk/controlRisk
        miRRList += [miRR]
        strokeRRList += [strokeRR]
        print(f"\t\tsimulation={i}, stroke RR={strokeRR:<4.2f}, mi RR={miRR:<4.2f}")
    print(f"average of {nSimulations} simulations: stroke RR={np.mean(strokeRRList):<4.2f}, mi RR={np.mean(miRRList):4.2f}")
    print(f"sd of {nSimulations} simulations: stroke RR={np.std(strokeRRList):<4.2f}, mi RR={np.std(miRRList):<4.2f}")
